In [43]:
%run base

In [52]:
# レコメンドタイプ3（類似ユーザー１）, レコメンドタイプ4（類似ユーザー２）
# 引数の設定
target_user_id=2
ref_users=1

# レコメンド商品数の設定
limit = app.config['LIMIT']

# SQLファイルの指定
sql_file = './app/static/sql/recommend_sql/which_users_buy_which_items.sql'

＃ 類似ユーザー抽出SQL（購買履歴）
```
SELECT
     c.user_id
    ,c.item_code
    ,CASE WHEN e.user_id is NULL THEN 0 ELSE 1 END flag
FROM (SELECT
             a.id as user_id
            ,b.item_code
       FROM
            users a, items b
       ORDER BY a.id, b.item_code) c
LEFT JOIN (SELECT
                 d.user_id
                ,d.item_code
            FROM
                order_histories d
            GROUP BY d.user_id, d.item_code) e
ON  c.user_id = e.user_id
AND c.item_code = e.item_code
```

In [60]:
# SQLファイルの読込とSQL実行
with codecs.open(sql_file, 'r', 'utf-8') as f:
    sql = f.read()
    df = pd_sql_to_df(sql)

In [61]:
# 縦持ちデータを横持ちデータに変換する
data0 = df.pivot_table(values=['flag'], index=['user_id'], columns=['item_code'], aggfunc='sum')
target_user_index = data0.index.get_loc(target_user_id)
data0.head()

flag                                     ...                      \
item_code  1   2   4   11  12  14  15  17  18  22  ... 897 898 899 900 901   
user_id                                            ...                       
1            1   0   1   0   0   0   0   0   0   1 ...   0   0   0   0   0   
2            0   1   0   1   0   1   0   0   0   0 ...   0   0   0   0   0   
3            0   0   1   0   0   1   0   0   1   0 ...   1   0   0   0   0   
4            0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   
5            0   0   0   1   0   0   0   0   0   0 ...   0   0   0   0   0   

                               
item_code 902 903 923 924 925  
user_id                        
1           0   0   0   0   0  
2           0   0   0   0   0  
3           0   1   0   0   0  
4           0   0   0   0   0  
5           0   0   0   0   0  

[5 rows x 155 columns]

In [62]:
# 相関行列からtarget_user_id行列の相関係数ベクトルの取得
data1 = np.corrcoef(data0)[target_user_index]
# data1

In [63]:
# 相関係数の高い順に並べる
data2 = (-data1).argsort()
#data2

In [64]:
# 参考ユーザーの選択（インデックス取得）
ref_user_ids = data2[1:ref_users+1]
#ref_user_ids

In [65]:
# 参考ユーザーの購入商品から対象ユーザーの購入商品を引く
recommend_items = select_items(ref_user_ids, target_user_id)

In [66]:
# 5商品を抽出する
recommend_items = recommend_items[0:limit]
recommend_items

[86, 414, 621, 4, 534]